## Load Libaries and Import necessary dataset

In [2]:
# install.packages(c("Rcpp", "rstan"))
# install.packages("prophet")

In [ ]:
# Load required libraries
library(readxl)
library(dplyr)
library(Rcpp)
# library(rstan)
# library(prophet)
install.packages("gt")
library("gt")
install.packages("ggplot2")
library(ggplot2)


Installing package into ‘/home/codespace/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)



Installing package into ‘/home/codespace/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

Warning message:
“dependency ‘gtable’ is not available”
Warning message in install.packages("ggplot2"):
“installation of package ‘ggplot2’ had non-zero exit status”


ERROR: Error in library(ggplot2): there is no package called ‘ggplot2’


In [ ]:

# Extract deposit_levels to use it in this file
deposit_levels <- readRDS("/workspaces/DUKE_MIDS_QFC_Final_Project/data_cleaning/deposit_levels.rds") # Load the data


## Data Exploration for Prophet Assumption

In [ ]:
glimpse(deposit_levels)

Rows: 162
Columns: 4
$ time     <chr> "1984Q1", "1984Q2", "1984Q3", "1984Q4", "1985Q1", "1985Q2", "…
$ year     <dbl> 1984, 1984, 1984, 1984, 1985, 1985, 1985, 1985, 1986, 1986, 1…
$ quarter  <dbl> 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1…
$ deposits <chr> "22917.257000000001", "23575.191999999999", "24308.965", "249…


### Data Regularity and Sufficient Historical Data Expectation

In [ ]:
any(is.na(deposit_levels)) # no missing values

[1] FALSE

> The time series has regular quaterly intervals and no missing values so the data regurlarity expectation is satified. 

> It also has decades worth of data so the "sufficient historical data" expectation is satisfied. 

### Presence of Trend Expectation

In [ ]:
library(ggplot2)

ggplot(deposit_levels, aes(x = as.Date(paste0(year, "-Q", quarter, "-01"), format = "%Y-Q%q-%d"), y = as.numeric(deposits))) +
  geom_line(color = "blue") +
  labs(title = "Quarterly Deposit Levels", x = "Date", y = "Deposits") +
  theme_minimal()


ERROR: Error in library(ggplot2): there is no package called ‘ggplot2’


### Outliers

### Seasonality

## Model Fit

In [ ]:
library(dplyr)

# Convert the data into Prophet's format
deposit_levels <- deposit_levels %>%
  mutate(
    ds = as.Date(paste0(year, "-Q", quarter, "-01"), format = "%Y-Q%q-%d"), # Convert year/quarter to date
    y = as.numeric(deposits) # Ensure deposits are numeric
  ) %>%
  select(ds, y)
glimpse(deposit_levels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in mutate(., ds = as.Date(paste0(year, "-Q", quarter, "-01"), format = "%Y-Q%q-%d"), : object 'deposit_levels' not found
